In [54]:
import pandas as pd
from models.Hermen.model_BE2 import model_BE2


FBS = pd.read_csv('models/Hermen/data/FBS.csv')
QC = pd.read_csv('models/Hermen/data/QC.csv').rename(columns={'FBS_item': 'Item'})
RL = pd.read_csv('models/Hermen/data/RL.csv').dropna(subset=['GGI_code'])

QC = QC.groupby(['Item', 'GGI_code', 'Year', 'Area']).mean().reset_index()

In [55]:
# Aux functions, to improve
def pivot(df):
    return df.pivot(index=['Item'], columns='GGI_code', values='Value')


def select_country_year(df, country, year):
    return df[(df.Area==country) & (df.Year == year)].reset_index(drop=True)


def df_to_dict(df):
    X = {}
    for code in df.columns:
        X[code] = df[code]
    return X

In [61]:
year = 2018
country = 'Germany'

FBS_country_year = select_country_year(FBS, country, year)
QC_country_year = select_country_year(QC, country, year)
RL_country_year = select_country_year(RL, country, year)

In [62]:
crop_group = FBS_country_year[FBS_country_year['group'] == 'crop'].Item

In [63]:
def FBS_QC_RL_to_X(FBS, QC, RL):
    
    X = {}
    
    FBS = df_to_dict(pivot(FBS))
    QC = df_to_dict(pivot(QC))
    RL = dict(zip(RL.GGI_code, RL.Value))
    
    print(RL.keys())
    for dictionnary in [FBS, QC, RL]:
        X.update(dictionnary)
        
        
    X['P'] = X['P'].dropna()
    return X

In [64]:
X = FBS_QC_RL_to_X(FBS_country_year, QC_country_year, RL_country_year)


# TO be added in a dataframe
X['FCRi'] = pd.Series([25, 6.4, 3.003, 2.3, 0.7, 15], index=['Bovine Meat', 'Pigmeat', 'Poultry Meat', 'Eggs', 'Milk - Excluding Butter', 'Meat, Other'])


# Define policy inputs

X['FWPPi'] = 0
X['FWCRi'] = 0
X['CRfd'] = 2
X['CD_corr'] = 3
X['R_rate'] = 0


# Can be fixed by policy or not
X['FMi'] = X['FDi'].loc[crop_group] / X['FDi'].sum()
X['TCLD_baseline'] = X['TCLDi'].sum()


dict_keys(['TLA', 'CL_t_minus_1', 'IL_t_minus_1', 'FL_t_minus_1'])


In [65]:
res = model_BE2.run(X)

In [66]:
res['BE2']

32.68361096800046